In [1]:
using CompTime, InteractiveUtils

In [2]:
function pw_rec(n, x)
    if iszero(n)
        one(x)
    else
        x * pw_rec(n - 1, x)
    end
end

pw_rec (generic function with 1 method)

In [3]:
pw_rec(3, "Abc")

"AbcAbcAbc"

In [4]:
function pw_loop(n, x)
    r = one(x)
    for i in 1:n
        r = r * x
    end
    return r
end

pw_loop (generic function with 1 method)

In [5]:
pw_loop(3, "Abc")

"AbcAbcAbc"

In [6]:
@ct_enable function pw_ct(@ct(n), x)
    r = one(x)
    @ct_ctrl for i in 1:n
        r = r * x
    end
    return r
end

runtime (generic function with 1 method)

In [7]:
pw_ct(n, x) = pw_ct(Val{n}, x)

pw_ct (generic function with 2 methods)

In [8]:
pw_ct(3, "Abc")

"AbcAbcAbc"

In [9]:
runtime(pw_ct, 3, "Abc")

"AbcAbcAbc"

In [10]:
comptime(pw_ct, Val{3}, "Abc")

"AbcAbcAbc"

In [11]:
@code_typed pw_ct(Val{3}, 10)

CodeInfo(
1 ─ %1 = Base.mul_int(1, x)::Int64
│   %2 = Base.mul_int(%1, x)::Int64
│   %3 = Base.mul_int(%2, x)::Int64
└──      return %3
) => Int64

In [12]:
# @code_typed (runtime(pw_ct, 3, 10))

In [13]:
debug(pw_ct, Val{3}, 10)

quote
    #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/pw.ipynb:1 =#
    #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/pw.ipynb:2 =#
    r = one(x)
    #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/pw.ipynb:3 =#
    begin
        begin
            #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/pw.ipynb:4 =#
            r = r * x
            #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/pw.ipynb:5 =#
        end
        begin
            #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/pw.ipynb:4 =#
            r = r * x
            #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/pw.ipynb:5 =#
        end
        begin
            #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/pw.ipynb:4 =#
            r = r * x
            #= /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/pw.ipynb:5 =#
        end
    end
    #= /home/roman/dev